In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os

1. Set hyper parameters - learning rate, training epochs, batch size, etc.
2. Make a data pipelining - tf.data
3. Build a neural network model - tf.keras functional APIs
4. Define a loss function - cross entropy
5. Calculate a gradient - tf.GradientTape
6. Select an optimizer - adam optimizer
7. Define a metric for model's performance - accuracy
8. (optional) Make a checkpoint for saving
9. Train and Validate a neural network model

In [2]:
# 1. Set hyper parameters

learning_rate = 0.001
training_epochs = 15
batch_size = 100

cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints' # checkpoint 저장할 디렉토리 설정
model_dir_name = 'mnist_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

In [3]:
# 2. Make a Data Pipelining

mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

## Limitation of Sequential API
- We can't make<br>
    Multi-input models<br>
    Multi-output models<br>
    Models with shared layers(the same layer called several times)<br>
    Models with non-sequential data flos(e.g.. Residual connections)<br>

In [21]:
# 3. Build a Neural Network Model

def create_model():
    inputs = keras.Input(shape=(28, 28, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, padding='SAME', activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D(padding='SAME')(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=3, padding='SAME', activation=tf.nn.relu)(pool1)
    pool2 = keras.layers.MaxPool2D(padding='SAME')(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=3, padding='SAME', activation=tf.nn.relu)(pool2)
    pool3 = keras.layers.MaxPool2D(padding='SAME')(conv3)
    pool3_flat = keras.layers.Flatten()(pool3)
    dense4 = keras.layers.Dense(units=256, activation=tf.nn.relu)(pool3_flat)
    drop4 = keras.layers.Dropout(rate=0.4)(dense4)
    logits = keras.layers.Dense(units=10)(drop4)
    return keras.Model(inputs=inputs, outputs=logits)

model = create_model()
model.summary() # Summary를 통해 만들어놓은 layer 형태 알 수 있음

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 4, 128)         0   

In [14]:
def create_model():
    inputs = keras.Input(shape=(28, 28, 256))
    conv1 = keras.layers.Conv2D(filters=64, kernel_size=1, padding='SAME', activation=keras.layers.ReLU())(inputs)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=3, padding='SAME', activation=keras.layers.ReLU())(conv1)
    conv3 = keras.layers.Conv2D(filters=256, kernel_size=1, padding='SAME', activation=keras.layers.ReLU())(conv2)
    add3 = keras.layers.add([conv3, inputs])
    relu3 = keras.layers.ReLU()(add3)
    model = keras.Model(inputs=inputs, outputs=relu3)
    return model
model = create_model()
model.summary() # Summary를 통해 만들어놓은 layer 형태 알 수 있음

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 256) 0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 28, 28, 64)   16448       input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 28, 28, 64)   36928       conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 28, 28, 256)  16640       conv2d_10[0][0]                  
____________________________________________________________________________________________

In [22]:
# 4. Define a Loss Function
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

In [23]:
# 5. Calculate a Gradient
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [24]:
# 6. Select an Optimizer
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

In [25]:
# 7. Define a Metric for Model's Performance
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [26]:
# 8. Make a Checkpoint for Saving
checkpoint = tf.train.Checkpoint(cnn=model)

In [27]:
# 9. Train and Validate a Neural Network Model
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        grads = grad(model, images, labels)
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:
        acc = evaluate(model, images, labels)
        avg_test_acc = avg_test_acc + acc
        test_step += 1
    avg_test_acc = avg_test_acc / test_step
    
    print('Epoch:', '{}'.format(epoch + 1), 'loss = {:.8f}'.format(avg_loss), 'train accuracy = {:.4f}'.format(avg_train_acc),
         'test accuracy = {:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)

Epoch: 1 loss = 0.17617741 train accuracy = 0.9564 test accuracy = 0.9839
Epoch: 2 loss = 0.04655004 train accuracy = 0.9899 test accuracy = 0.9881
Epoch: 3 loss = 0.03387206 train accuracy = 0.9928 test accuracy = 0.9909
Epoch: 4 loss = 0.02406434 train accuracy = 0.9953 test accuracy = 0.9915
Epoch: 5 loss = 0.01876381 train accuracy = 0.9967 test accuracy = 0.9919
Epoch: 6 loss = 0.01495505 train accuracy = 0.9972 test accuracy = 0.9942
Epoch: 7 loss = 0.01259666 train accuracy = 0.9979 test accuracy = 0.9918
Epoch: 8 loss = 0.01146172 train accuracy = 0.9982 test accuracy = 0.9922
Epoch: 9 loss = 0.00966463 train accuracy = 0.9985 test accuracy = 0.9938
Epoch: 10 loss = 0.00739531 train accuracy = 0.9990 test accuracy = 0.9913
Epoch: 11 loss = 0.00732057 train accuracy = 0.9990 test accuracy = 0.9935
Epoch: 12 loss = 0.00512403 train accuracy = 0.9995 test accuracy = 0.9938
Epoch: 13 loss = 0.00615596 train accuracy = 0.9992 test accuracy = 0.9939
Epoch: 14 loss = 0.00442466 train 